In [3]:
import time
import sqlite3
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
import import_ipynb
import utils 

# Scrape data using Selenium
def scrape_tee_times(url, course_ids):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)
    
    date_element = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="teetimes-header-date"] button')
    date_text = date_element.text.strip()

    tee_times = driver.find_elements(By.CSS_SELECTOR, 'div.MuiCardContent-root')
    scraped_data = []

    for index, tee_time in enumerate(tee_times):
        try:
            course_id = course_ids[index % len(course_ids)]
            course_name_element = tee_time.find_element(By.CSS_SELECTOR, 'p[data-testid="teetimes-tile-course-name"]').text
            time_element = tee_time.find_element(By.CSS_SELECTOR, 'p[data-testid="teetimes-tile-time"]').text
            players_text = tee_time.find_element(By.CSS_SELECTOR, 'p[data-testid="teetimes-tile-available-players"]').text
            players_number = "".join(filter(str.isdigit, players_text))
            holes_element = tee_time.find_element(By.CSS_SELECTOR, 'p[data-testid="teetimes-tile-hole-verbiage"]').text
            price_elements = tee_time.find_elements(By.CSS_SELECTOR, 'p.css-1m88q2v')
            prices = ", ".join([price.text for price in price_elements])

            scraped_data.append((date_text, course_id, course_name_element, time_element, players_number, holes_element, prices))
        except Exception as e:
            print(f"Error extracting data for a tee time: {e}")

    driver.quit()
    return scraped_data

# Main execution
def get_hyland_hills_tee_time():
    scrape_url = f"https://hyland-hills-park-recreation-district.book.teeitup.com/?course=9201,9202,9203&date={datetime.now().strftime('%Y-%m-%d')}&max=9999"
    course_ids = [9201, 9202, 9203]
    scraped_data = scrape_tee_times(scrape_url, course_ids)
    
    if scraped_data:
        utils.update_database(scraped_data, table_name="TeeTimes")
        print("Hyland Hills data has been updated in the SQLite database 'GolfteeTimes_database.db'.")
    else:
        print("No tee time data scraped from Hyland Hills.")
    
    return scraped_data  # Return the data for use in utils.main()

if __name__ == "__main__":
    utils.initialize_database()
    get_hyland_hills_tee_time()


No tee time data scraped from Hyland Hills.
